In [102]:
import pandas as pd
from datetime import datetime as dt

In [103]:
data = pd.read_csv('subscribers_new.csv', parse_dates=['account_creation_date'])

In [104]:
data.head()

,Unnamed: 0,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,age,male_TF,...,join_fee,language,payment_type,num_trial_days,current_sub_TF,payment_period,last_payment,next_payment,cancel_date,trial_completed
0,162502,21724479,economy,NaN,comedy,access to exclusive content,NaN,NaN,NaN,False,...,NaN,ar,Standard Charter,14,True,1,2020-02-07 21:44:16,2020-06-07 21:44:16,NaN,True
1,380374,23383224,base,NaN,comedy,access to exclusive content,22.951579,NaN,70.0,False,...,0.3303,ar,NaN,14,True,1,2020-03-15 15:44:35,2020-07-15 15:44:35,NaN,True
2,496617,26844789,enhanced,3.0,regional,replace OTT,36.001186,2.0,25.0,True,...,0.1101,ar,NaN,14,False,0,NaN,NaN,NaN,True
3,356799,29417030,base,NaN,drama,replace OTT,20.051667,NaN,30.0,False,...,0.0367,ar,NaN,14,False,0,NaN,NaN,NaN,True
4,681701,26723159,base,4.0,comedy,replace OTT,22.951579,3.0,28.0,False,...,0.0367,ar,NaN,14,True,2,2020-02-19 12:57:07,2020-06-19 12:57:07,NaN,True


In [105]:
data.shape

(227628, 31)

In [106]:
data = data[(data['payment_period']>=1)&(data['trial_completed'] == True)]

In [107]:
df = data[['attribution_technical','attribution_survey','account_creation_date']]

In [108]:
df.head()

,attribution_technical,attribution_survey,account_creation_date
0,facebook,facebook,2020-01-24 21:44:16
1,facebook,facebook,2020-03-01 15:44:35
4,discovery,youtube,2019-10-05 12:57:07
5,bing,tv,2020-03-03 20:15:43
6,bing,search,2020-02-19 18:30:15


In [109]:
df.shape

(131968, 3)

In [110]:
df['year'] = df['account_creation_date'].dt.year

<ipython-input-110-ede932d59b48>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['account_creation_date'].dt.year


In [111]:
df['month'] = df['account_creation_date'].dt.month

<ipython-input-111-c76ef83fde34>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = df['account_creation_date'].dt.month


In [112]:
df.head()

,attribution_technical,attribution_survey,account_creation_date,year,month
0,facebook,facebook,2020-01-24 21:44:16,2020,1
1,facebook,facebook,2020-03-01 15:44:35,2020,3
4,discovery,youtube,2019-10-05 12:57:07,2019,10
5,bing,tv,2020-03-03 20:15:43,2020,3
6,bing,search,2020-02-19 18:30:15,2020,2


In [113]:
df = df.dropna()

In [114]:
df = df.drop(columns=['account_creation_date'])

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130884 entries, 0 to 227626
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   attribution_technical  130884 non-null  object
 1   attribution_survey     130884 non-null  object
 2   year                   130884 non-null  int64 
 3   month                  130884 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 5.0+ MB


In [116]:
df1 = df.reset_index().drop(columns = 'index')

In [117]:
df1

,attribution_technical,attribution_survey,year,month
0,facebook,facebook,2020,1
1,facebook,facebook,2020,3
2,discovery,youtube,2019,10
3,bing,tv,2020,3
4,bing,search,2020,2
...,...,...,...,...
130879,email,pinterest,2020,1
130880,facebook,facebook_organic,2019,11
130881,google_organic,referral,2019,12
130882,facebook,facebook,2019,12


In [120]:
df1['channel'] = ''
for i in range(130884):
    if 'sem' in df1['attribution_technical'][i]:
        df1.loc[i,'channel'] = 'search'
    else:
        df1.loc[i,'channel'] = df1['attribution_technical'][i]

In [121]:
df1

,attribution_technical,attribution_survey,year,month,channel
0,facebook,facebook,2020,1,facebook
1,facebook,facebook,2020,3,facebook
2,discovery,youtube,2019,10,discovery
3,bing,tv,2020,3,bing
4,bing,search,2020,2,bing
...,...,...,...,...,...
130879,email,pinterest,2020,1,email
130880,facebook,facebook_organic,2019,11,facebook
130881,google_organic,referral,2019,12,google_organic
130882,facebook,facebook,2019,12,facebook


In [122]:
df_new = df1['channel'].groupby([df1['year'], df1['month']]).value_counts()

In [123]:
df_new

year  month  channel          
2019  6      facebook             224
             search                76
             email                 56
             organic               38
             email_blast           32
                                 ... 
2020  3      appstore               9
             influencer             2
             pinterest_organic      2
             content_greatist       1
             direct_mail            1
Name: channel, Length: 228, dtype: int64

In [125]:
df_comp = pd.DataFrame({'count':df_new})

In [126]:
df_comp.reset_index(inplace=True)

In [127]:
df_comp

,year,month,channel,count
0,2019,6,facebook,224
1,2019,6,search,76
2,2019,6,email,56
3,2019,6,organic,38
4,2019,6,email_blast,32
...,...,...,...,...
223,2020,3,appstore,9
224,2020,3,influencer,2
225,2020,3,pinterest_organic,2
226,2020,3,content_greatist,1


In [128]:
df_comp=df_comp[~df_comp['month'].isin([6])]
df_comp=df_comp[~df_comp['month'].isin([3])]

In [129]:
df_comp

,year,month,channel,count
17,2019,7,facebook,2004
18,2019,7,search,1061
19,2019,7,email,796
20,2019,7,organic,582
21,2019,7,email_blast,415
...,...,...,...,...
198,2020,2,direct_mail,15
199,2020,2,appstore,6
200,2020,2,influencer,3
201,2020,2,playstore,3


In [56]:
#df_comp = df_comp.drop(columns = 'attribution_technical')

In [130]:
data2 = pd.read_csv('channel_spend_graduate.csv')

In [131]:
data2.head()

,channel,date,spend_AED
0,bing,201907,400
1,display,201907,12
2,facebook,201907,9000
3,search,201907,13000
4,youtube,201907,90


In [132]:
data2['date']= pd.to_datetime(data2['date'], format='%Y%m')

In [133]:
data2['year'] = data2['date'].dt.year

In [134]:
data2['month'] = data2['date'].dt.month

In [135]:
data2 = data2.drop(columns='date')

In [136]:
data2

,channel,spend_AED,year,month
0,bing,400,2019,7
1,display,12,2019,7
2,facebook,9000,2019,7
3,search,13000,2019,7
4,youtube,90,2019,7
5,bing,900,2019,8
6,display,20,2019,8
7,facebook,13000,2019,8
8,search,18500,2019,8
9,youtube,180,2019,8


In [137]:
df_combined = pd.merge(df_comp, data2, how='left', on=['year','month','channel'])
df_combined.shape

(186, 5)

In [138]:
df_combined['CAC'] = df_combined['spend_AED']/df_combined['count']

In [139]:
df_combined

,year,month,channel,count,spend_AED,CAC
0,2019,7,facebook,2004,9000.0,4.491018
1,2019,7,search,1061,13000.0,12.252592
2,2019,7,email,796,NaN,NaN
3,2019,7,organic,582,NaN,NaN
4,2019,7,email_blast,415,NaN,NaN
...,...,...,...,...,...,...
181,2020,2,direct_mail,15,NaN,NaN
182,2020,2,appstore,6,NaN,NaN
183,2020,2,influencer,3,NaN,NaN
184,2020,2,playstore,3,NaN,NaN


In [142]:
b = df_combined.dropna().groupby('channel')

In [143]:
facebook_comp = b.get_group('facebook').sort_values(by='spend_AED')
facebook_comp

,year,month,channel,count,spend_AED,CAC
0,2019,7,facebook,2004,9000.0,4.491018
42,2019,9,facebook,3726,10500.0,2.818035
137,2020,1,facebook,4096,11000.0,2.685547
21,2019,8,facebook,4809,13000.0,2.703265
161,2020,2,facebook,4893,14000.0,2.861230
112,2019,12,facebook,6026,16000.0,2.655161
64,2019,10,facebook,6264,17000.0,2.713921
87,2019,11,facebook,8459,23000.0,2.718998


In [144]:
facebook_comp['marginal_count'] = 0
facebook_comp['marginal_cost'] = 0
for i in range(0,len(facebook_comp)):
    if i == 0:
        facebook_comp['marginal_count'].values[i] = facebook_comp['count'].values[i]
        facebook_comp['marginal_cost'].values[i] = facebook_comp['spend_AED'].values[i]
    else:
        facebook_comp['marginal_count'].values[i] = facebook_comp['count'].values[i] - facebook_comp['count'].values[i-1]
        facebook_comp['marginal_cost'].values[i] = facebook_comp['spend_AED'].values[i] - facebook_comp['spend_AED'].values[i-1]
facebook_comp

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost
0,2019,7,facebook,2004,9000.0,4.491018,2004,9000
42,2019,9,facebook,3726,10500.0,2.818035,1722,1500
137,2020,1,facebook,4096,11000.0,2.685547,370,500
21,2019,8,facebook,4809,13000.0,2.703265,713,2000
161,2020,2,facebook,4893,14000.0,2.861230,84,1000
112,2019,12,facebook,6026,16000.0,2.655161,1133,2000
64,2019,10,facebook,6264,17000.0,2.713921,238,1000
87,2019,11,facebook,8459,23000.0,2.718998,2195,6000


In [145]:
facebook_comp['marginal CAC'] = facebook_comp['marginal_cost']/facebook_comp['marginal_count']
facebook_comp

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost,marginal CAC
0,2019,7,facebook,2004,9000.0,4.491018,2004,9000,4.491018
42,2019,9,facebook,3726,10500.0,2.818035,1722,1500,0.871080
137,2020,1,facebook,4096,11000.0,2.685547,370,500,1.351351
21,2019,8,facebook,4809,13000.0,2.703265,713,2000,2.805049
161,2020,2,facebook,4893,14000.0,2.861230,84,1000,11.904762
112,2019,12,facebook,6026,16000.0,2.655161,1133,2000,1.765225
64,2019,10,facebook,6264,17000.0,2.713921,238,1000,4.201681
87,2019,11,facebook,8459,23000.0,2.718998,2195,6000,2.733485


In [146]:
search_comp = b.get_group('search').sort_values(by='spend_AED')
search_comp

,year,month,channel,count,spend_AED,CAC
1,2019,7,search,1061,13000.0,12.252592
22,2019,8,search,2258,18500.0,8.193091
43,2019,9,search,2337,19000.0,8.130081
65,2019,10,search,2883,24000.0,8.324662
88,2019,11,search,3380,25000.0,7.396450
113,2019,12,search,4327,38000.0,8.782066
136,2020,1,search,4735,41000.0,8.658923
162,2020,2,search,4702,44000.0,9.357720


In [147]:
search_comp['marginal_count'] = 0
search_comp['marginal_cost'] = 0
for i in range(0,len(search_comp)):
    if i == 0:
        search_comp['marginal_count'].values[i] = search_comp['count'].values[i]
        search_comp['marginal_cost'].values[i] = search_comp['spend_AED'].values[i]
    else:
        search_comp['marginal_count'].values[i] = search_comp['count'].values[i] - search_comp['count'].values[i-1]
        search_comp['marginal_cost'].values[i] = search_comp['spend_AED'].values[i] - search_comp['spend_AED'].values[i-1]

In [148]:
search_comp['marginal CAC'] = search_comp['marginal_cost']/search_comp['marginal_count']
search_comp

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost,marginal CAC
1,2019,7,search,1061,13000.0,12.252592,1061,13000,12.252592
22,2019,8,search,2258,18500.0,8.193091,1197,5500,4.594820
43,2019,9,search,2337,19000.0,8.130081,79,500,6.329114
65,2019,10,search,2883,24000.0,8.324662,546,5000,9.157509
88,2019,11,search,3380,25000.0,7.396450,497,1000,2.012072
113,2019,12,search,4327,38000.0,8.782066,947,13000,13.727561
136,2020,1,search,4735,41000.0,8.658923,408,3000,7.352941
162,2020,2,search,4702,44000.0,9.357720,-33,3000,-90.909091


In [149]:
display_comp = b.get_group('display').sort_values(by='spend_AED')
display_comp

,year,month,channel,count,spend_AED,CAC
16,2019,7,display,7,12.0,1.714286
60,2019,9,display,8,13.0,1.625000
79,2019,10,display,28,19.0,0.678571
35,2019,8,display,33,20.0,0.606061
100,2019,11,display,64,29.0,0.453125
126,2019,12,display,65,31.0,0.476923
171,2020,2,display,189,94.0,0.497354
144,2020,1,display,325,148.0,0.455385


In [150]:
display_comp['marginal_count'] = 0
display_comp['marginal_cost'] = 0
for i in range(0,len(display_comp)):
    if i == 0:
        display_comp['marginal_count'].values[i] = display_comp['count'].values[i]
        display_comp['marginal_cost'].values[i] = display_comp['spend_AED'].values[i]
    else:
        display_comp['marginal_count'].values[i] = display_comp['count'].values[i] - display_comp['count'].values[i-1]
        display_comp['marginal_cost'].values[i] = display_comp['spend_AED'].values[i] - display_comp['spend_AED'].values[i-1]

In [151]:
display_comp['marginal CAC'] = display_comp['marginal_cost']/display_comp['marginal_count']
display_comp

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost,marginal CAC
16,2019,7,display,7,12.0,1.714286,7,12,1.714286
60,2019,9,display,8,13.0,1.625000,1,1,1.000000
79,2019,10,display,28,19.0,0.678571,20,6,0.300000
35,2019,8,display,33,20.0,0.606061,5,1,0.200000
100,2019,11,display,64,29.0,0.453125,31,9,0.290323
126,2019,12,display,65,31.0,0.476923,1,2,2.000000
171,2020,2,display,189,94.0,0.497354,124,63,0.508065
144,2020,1,display,325,148.0,0.455385,136,54,0.397059


In [152]:
bing_comp = b.get_group('bing').sort_values(by='spend_AED')
bing_comp

,year,month,channel,count,spend_AED,CAC
130,2019,12,bing,13,300.0,23.076923
13,2019,7,bing,24,400.0,16.666667
36,2019,8,bing,30,900.0,30.000000
55,2019,9,bing,51,1000.0,19.607843
78,2019,10,bing,53,1100.0,20.754717
102,2019,11,bing,59,1300.0,22.033898
150,2020,1,bing,93,2100.0,22.580645
172,2020,2,bing,179,3700.0,20.670391


In [153]:
bing_comp['marginal_count'] = 0
bing_comp['marginal_cost'] = 0
for i in range(0,len(bing_comp)):
    if i == 0:
        bing_comp['marginal_count'].values[i] = bing_comp['count'].values[i]
        bing_comp['marginal_cost'].values[i] = bing_comp['spend_AED'].values[i]
    else:
        bing_comp['marginal_count'].values[i] = bing_comp['count'].values[i] - bing_comp['count'].values[i-1]
        bing_comp['marginal_cost'].values[i] = bing_comp['spend_AED'].values[i] - bing_comp['spend_AED'].values[i-1]

In [154]:
bing_comp['marginal CAC'] = bing_comp['marginal_cost']/bing_comp['marginal_count']
bing_comp

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost,marginal CAC
130,2019,12,bing,13,300.0,23.076923,13,300,23.076923
13,2019,7,bing,24,400.0,16.666667,11,100,9.090909
36,2019,8,bing,30,900.0,30.000000,6,500,83.333333
55,2019,9,bing,51,1000.0,19.607843,21,100,4.761905
78,2019,10,bing,53,1100.0,20.754717,2,100,50.000000
102,2019,11,bing,59,1300.0,22.033898,6,200,33.333333
150,2020,1,bing,93,2100.0,22.580645,34,800,23.529412
172,2020,2,bing,179,3700.0,20.670391,86,1600,18.604651


In [155]:
youtube_comp = b.get_group('youtube').sort_values(by='spend_AED')
youtube_comp

,year,month,channel,count,spend_AED,CAC
20,2019,7,youtube,1,90.0,90.000000
63,2019,9,youtube,1,100.0,100.000000
82,2019,10,youtube,7,130.0,18.571429
38,2019,8,youtube,11,180.0,16.363636
103,2019,11,youtube,36,550.0,15.277778
127,2019,12,youtube,55,900.0,16.363636
174,2020,2,youtube,159,2420.0,15.220126
145,2020,1,youtube,312,4360.0,13.974359


In [156]:
youtube_comp['marginal_count'] = 0
youtube_comp['marginal_cost'] = 0
for i in range(0,len(youtube_comp)):
    if i == 0:
        youtube_comp['marginal_count'].values[i] = youtube_comp['count'].values[i]
        youtube_comp['marginal_cost'].values[i] = youtube_comp['spend_AED'].values[i]
    else:
        youtube_comp['marginal_count'].values[i] = youtube_comp['count'].values[i] - youtube_comp['count'].values[i-1]
        youtube_comp['marginal_cost'].values[i] = youtube_comp['spend_AED'].values[i] - youtube_comp['spend_AED'].values[i-1]

In [157]:
youtube_comp['marginal CAC'] = youtube_comp['marginal_cost']/youtube_comp['marginal_count']
youtube_comp

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost,marginal CAC
20,2019,7,youtube,1,90.0,90.000000,1,90,90.000000
63,2019,9,youtube,1,100.0,100.000000,0,10,inf
82,2019,10,youtube,7,130.0,18.571429,6,30,5.000000
38,2019,8,youtube,11,180.0,16.363636,4,50,12.500000
103,2019,11,youtube,36,550.0,15.277778,25,370,14.800000
127,2019,12,youtube,55,900.0,16.363636,19,350,18.421053
174,2020,2,youtube,159,2420.0,15.220126,104,1520,14.615385
145,2020,1,youtube,312,4360.0,13.974359,153,1940,12.679739


### Attribution - Survey

In [160]:
df_new2 = df['attribution_survey'].groupby([df['year'], df['month']]).value_counts()
df_new2

year  month  attribution_survey
2019  6      facebook              260
             referral              110
             tv                     47
             pinterest              33
             other                  25
                                  ... 
2020  3      affiliate             124
             sirius xm              74
             facebook_organic       73
             amfm_radio             60
             direct_mail            12
Name: attribution_survey, Length: 147, dtype: int64

In [161]:
df_user = pd.DataFrame({'count':df_new2})
df_user.reset_index(inplace=True)
df_user

,year,month,attribution_survey,count
0,2019,6,facebook,260
1,2019,6,referral,110
2,2019,6,tv,47
3,2019,6,pinterest,33
4,2019,6,other,25
...,...,...,...,...
142,2020,3,affiliate,124
143,2020,3,sirius xm,74
144,2020,3,facebook_organic,73
145,2020,3,amfm_radio,60


In [162]:
df_user.columns = ['year','month','channel','count']

In [163]:
df_user = df_user[~df_user['month'].isin([6])]
df_user = df_user[~df_user['month'].isin([3])]

In [164]:
df_user

,year,month,channel,count
12,2019,7,facebook,2754
13,2019,7,referral,1577
14,2019,7,tv,603
15,2019,7,pinterest,527
16,2019,7,other,339
...,...,...,...,...
126,2020,2,affiliate,259
127,2020,2,sirius xm,140
128,2020,2,facebook_organic,129
129,2020,2,amfm_radio,91


In [165]:
df2 = pd.merge(df_user, data2, how='left', on=['year','month','channel'])

In [166]:
df2.shape

(119, 5)

In [167]:
df2['CAC'] = df2['spend_AED']/df2['count']

In [168]:
df2.dropna()

,year,month,channel,count,spend_AED,CAC
0,2019,7,facebook,2754,9000.0,3.267974
5,2019,7,search,282,13000.0,46.099291
10,2019,7,youtube,40,90.0,2.250000
14,2019,8,facebook,7508,13000.0,1.731486
19,2019,8,search,544,18500.0,34.007353
22,2019,8,youtube,204,180.0,0.882353
28,2019,9,facebook,6939,10500.0,1.513186
31,2019,9,search,633,19000.0,30.015798
36,2019,9,youtube,166,100.0,0.602410
42,2019,10,facebook,8620,17000.0,1.972158


In [169]:
a = df2.dropna().groupby('channel')

In [170]:
for key, value in a:
    print(key)
    print(value)
    print(type(value))

facebook
     year  month   channel  count  spend_AED       CAC
0    2019      7  facebook   2754     9000.0  3.267974
14   2019      8  facebook   7508    13000.0  1.731486
28   2019      9  facebook   6939    10500.0  1.513186
42   2019     10  facebook   8620    17000.0  1.972158
57   2019     11  facebook  10816    23000.0  2.126479
72   2019     12  facebook   8568    16000.0  1.867414
87   2020      1  facebook   6451    11000.0  1.705162
103  2020      2  facebook   7263    14000.0  1.927578
<class 'pandas.core.frame.DataFrame'>
search
     year  month channel  count  spend_AED        CAC
5    2019      7  search    282    13000.0  46.099291
19   2019      8  search    544    18500.0  34.007353
31   2019      9  search    633    19000.0  30.015798
47   2019     10  search    664    24000.0  36.144578
60   2019     11  search    678    25000.0  36.873156
75   2019     12  search    832    38000.0  45.673077
90   2020      1  search    924    41000.0  44.372294
106  2020      2  s

In [171]:
facebook_user = a.get_group('facebook').sort_values(by='spend_AED')
facebook_user

,year,month,channel,count,spend_AED,CAC
0,2019,7,facebook,2754,9000.0,3.267974
28,2019,9,facebook,6939,10500.0,1.513186
87,2020,1,facebook,6451,11000.0,1.705162
14,2019,8,facebook,7508,13000.0,1.731486
103,2020,2,facebook,7263,14000.0,1.927578
72,2019,12,facebook,8568,16000.0,1.867414
42,2019,10,facebook,8620,17000.0,1.972158
57,2019,11,facebook,10816,23000.0,2.126479


In [172]:
facebook_user['marginal_count'] = 0
facebook_user['marginal_cost'] = 0
for i in range(0,len(facebook_user)):
    if i == 0:
        facebook_user['marginal_count'].values[i] = facebook_user['count'].values[i]
        facebook_user['marginal_cost'].values[i] = facebook_user['spend_AED'].values[i]
    else:
        facebook_user['marginal_count'].values[i] = facebook_user['count'].values[i] - facebook_user['count'].values[i-1]
        facebook_user['marginal_cost'].values[i] = facebook_user['spend_AED'].values[i] - facebook_user['spend_AED'].values[i-1]

In [173]:
facebook_user['marginal CAC'] = facebook_user['marginal_cost']/facebook_user['marginal_count']
facebook_user

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost,marginal CAC
0,2019,7,facebook,2754,9000.0,3.267974,2754,9000,3.267974
28,2019,9,facebook,6939,10500.0,1.513186,4185,1500,0.358423
87,2020,1,facebook,6451,11000.0,1.705162,-488,500,-1.024590
14,2019,8,facebook,7508,13000.0,1.731486,1057,2000,1.892148
103,2020,2,facebook,7263,14000.0,1.927578,-245,1000,-4.081633
72,2019,12,facebook,8568,16000.0,1.867414,1305,2000,1.532567
42,2019,10,facebook,8620,17000.0,1.972158,52,1000,19.230769
57,2019,11,facebook,10816,23000.0,2.126479,2196,6000,2.732240


In [174]:
search_user = a.get_group('search').sort_values(by='spend_AED')
search_user

,year,month,channel,count,spend_AED,CAC
5,2019,7,search,282,13000.0,46.099291
19,2019,8,search,544,18500.0,34.007353
31,2019,9,search,633,19000.0,30.015798
47,2019,10,search,664,24000.0,36.144578
60,2019,11,search,678,25000.0,36.873156
75,2019,12,search,832,38000.0,45.673077
90,2020,1,search,924,41000.0,44.372294
106,2020,2,search,1063,44000.0,41.392286


In [175]:
search_user['marginal_count'] = 0
search_user['marginal_cost'] = 0
for i in range(0,len(search_user)):
    if i == 0:
        search_user['marginal_count'].values[i] = search_user['count'].values[i]
        search_user['marginal_cost'].values[i] = search_user['spend_AED'].values[i]
    else:
        search_user['marginal_count'].values[i] = search_user['count'].values[i] - search_user['count'].values[i-1]
        search_user['marginal_cost'].values[i] = search_user['spend_AED'].values[i] - search_user['spend_AED'].values[i-1]

In [176]:
search_user['marginal CAC'] = search_user['marginal_cost']/search_user['marginal_count']
search_user

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost,marginal CAC
5,2019,7,search,282,13000.0,46.099291,282,13000,46.099291
19,2019,8,search,544,18500.0,34.007353,262,5500,20.992366
31,2019,9,search,633,19000.0,30.015798,89,500,5.617978
47,2019,10,search,664,24000.0,36.144578,31,5000,161.290323
60,2019,11,search,678,25000.0,36.873156,14,1000,71.428571
75,2019,12,search,832,38000.0,45.673077,154,13000,84.415584
90,2020,1,search,924,41000.0,44.372294,92,3000,32.608696
106,2020,2,search,1063,44000.0,41.392286,139,3000,21.582734


In [177]:
youtube_user = a.get_group('youtube').sort_values(by='spend_AED')
youtube_user

,year,month,channel,count,spend_AED,CAC
10,2019,7,youtube,40,90.0,2.250000
36,2019,9,youtube,166,100.0,0.602410
51,2019,10,youtube,218,130.0,0.596330
22,2019,8,youtube,204,180.0,0.882353
65,2019,11,youtube,252,550.0,2.182540
81,2019,12,youtube,200,900.0,4.500000
110,2020,2,youtube,462,2420.0,5.238095
92,2020,1,youtube,630,4360.0,6.920635


In [178]:
youtube_user['marginal_count'] = 0
youtube_user['marginal_cost'] = 0
for i in range(0,len(search_user)):
    if i == 0:
        youtube_user['marginal_count'].values[i] = youtube_user['count'].values[i]
        youtube_user['marginal_cost'].values[i] = youtube_user['spend_AED'].values[i]
    else:
        youtube_user['marginal_count'].values[i] = youtube_user['count'].values[i] - youtube_user['count'].values[i-1]
        youtube_user['marginal_cost'].values[i] = youtube_user['spend_AED'].values[i] - youtube_user['spend_AED'].values[i-1]

In [179]:
youtube_user['marginal CAC'] = youtube_user['marginal_cost']/youtube_user['marginal_count']
youtube_user

,year,month,channel,count,spend_AED,CAC,marginal_count,marginal_cost,marginal CAC
10,2019,7,youtube,40,90.0,2.250000,40,90,2.250000
36,2019,9,youtube,166,100.0,0.602410,126,10,0.079365
51,2019,10,youtube,218,130.0,0.596330,52,30,0.576923
22,2019,8,youtube,204,180.0,0.882353,-14,50,-3.571429
65,2019,11,youtube,252,550.0,2.182540,48,370,7.708333
81,2019,12,youtube,200,900.0,4.500000,-52,350,-6.730769
110,2020,2,youtube,462,2420.0,5.238095,262,1520,5.801527
92,2020,1,youtube,630,4360.0,6.920635,168,1940,11.547619


In [180]:
frames = [youtube_comp,youtube_user,facebook_comp,facebook_user,search_comp,search_user,display_comp,bing_comp]
attributions = pd.concat(frames)

In [181]:
attributions.to_csv('attributions.csv')